# Radiotherapy to OMOP

IDEA4RC radiotherapy is mainly mapped to OMOP Episode, procedure occurrence, measurement and observation following https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=744219820#gid=744219820

What should I do with `Radiotherapy performed location` in the `radiotherapy` table since I don't know how this information can be retrieved?

Some measurement concept id would be missing if we use the code as value as a code. Perhaps some of them should be observations instead?

Why we need treatmentResponse if it already is in episode event?

In [2]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [1]:
radiotherapy_IDEA4RC = pd.read_csv("./IDEA4RC-data/radiotherapyIDEA4RC.csv")
radiotherapy_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**config)
curIDEA = conn.cursor()

Radiotherapy performed location???
What to do with it?

### Radiotherapy to Episode

In [ ]:
df_tables=radiotherapy_IDEA4RC
sql = """
INSERT INTO EPISODE (episode_id, episode_start_date, episode_end_date, episode_concept_id,episode_type_concept_id)
VALUES (%s, %s, %s, %s,%s)
ON DUPLICATE KEY UPDATE
    episode_start_date = VALUES(episode_start_date),
    episode_end_date = VALUES(episode_end_date),
    episode_concept_id = VALUES(episode_concept_id),
    episode_type_concept_id=VALUES(episode_type_concept_id);
"""
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
for idx, row in df_tables.iterrows():
    cur.execute(sql,(row['episode_event'], row['startdate'], row['enddate'],row['treatmentResponse'],row['treatmentResponse']))
    conn.commit()

### Radiotherapy to Procedure Ocurrence

In [ ]:
queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s)
"""

sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_type_concept_id)
    VALUES (%s, %s, %s,%s,%s)
"""
def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_concept_id=row['Surgery type']
    procedure_date=row['Date of surgery']
    procedure_type_concept_id=0 #Still dont know what to do with this one
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    toTableEpisodeEventProcedureOcurrence(row['Episode Event reference'],newId)
    cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_type_concept_id))
    conn.commit()

### Radiotherapy to Measurement

Measurement concept id???

In [ ]:
sqlMeasuementNumber = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_number,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s,%s)
    """
measurement_type_concept_id=38000280
measurement_concept_id=0
columns=[
    "primaryOnly",
    "neckOnly",
    "primaryAndIpsilateralNeck",
    "primaryAndBilateralNeck",
    "distantMetastasis",
    "lung",
    "mediastinum",
    "bone",
    "softTissue",
    "liver",
    "treatmentCompletedAsPlanned",
    "beamQuality",
    "otherSpecify",
    "treatmentTechnique",
    "totalDoseGy",
    "numberOfFractions",
    "adaptiveRt",
    "igrt"
]

booleans=[
    "adaptiveRt",
    "igrt",
    "primaryOnly",
    "neckOnly",
    "primaryAndIpsilateralNeck",
    "primaryAndBilateralNeck",
    "distantMetastasis",
    "lung",
    "mediastinum",
    "bone",
    "softTissue",
    "liver"
]
boolValues=[4188539,4188540]
sqlCode="""
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_concept_id,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
for idx, row in df_tables.iterrows():
    for column in columns:
        curIDEA.execute(query, (row['Episode Event Reference']))
        res=curIDEA.fetchone()
        person_id=res
        measurement_concept_id= (column=='numberOfFractions') ? 4037631:0 #Should we use the modifier? is this value clear?
        measurement_value=row[column]
        date=row['startDate']
        if column=='totalDoseGy' or column=='numberOfFractions':
            value_as_number=measurement_value
            cur.execute(sqlMeasuementNumber,(person_id, measurement_concept_id, date, measurement_type_concept_id,value_as_number,measurement_event_id,CODIGO QUE HAY QUE VER)) WE NEED THE CODE
            conn.commit()
        else:
            if(column in booleans):
                if(measurement_value not in boolValues):
                    measurement_value=measurement_value==0?4188540:4188539
                
            cur.execute(sqlCode, (person_id, measurement_concept_id, date, measurement_type_concept_id,measurement_value,measurement_event_id, CODIGO QUE FALTA SABER))
            conn.commit()

### Radiotherapy to Observation

In [ ]:
sql="""
    INSERT INTO omopcdm.observation (person_id, observation_concept_id, observation_date, observation_type_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
observation_type_concept_id=37117814
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    observation_concept_id=row['intent']
    observation_date=row['startDate']    
    cur.execute(sql,(person_id, observation_concept_id, observation_date, observation_type_concept_id))
    conn.commit()